# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

### Решение

#### 1 Общая информация

In [ ]:
import pandas as pd

hdf = pd.read_csv('./horse_data.csv', header=None, names=['had_surgery', 'age', 'hospital_number', 'rectal_temp', 'pulse', 'respiratory_rate',
'extremities_temp', 'peripheral_pulse', 'mucous_membranes', 'capillary_refill_time', 'pain',
'peristalsis', 'abdominal_distension', 'nasogastric_tube', 'nasogastric_reflux',
'nasogastric_reflux_PH', 'rectal_examination', 'abdomen', 'packed_cell_volume', 'total_protein',
'abdominocentesis_appearance', 'abdomcentesis_total_protein', 'outcome', 'surgical_lesion',
'type_of_lesion1', 'type_of_lesion2', 'type_of_lesion3', 'cp_data'],
                  na_values='?', 
                  converters = {'type_of_lesion1' : str,
                                'type_of_lesion2' : str,
                                'type_of_lesion3' : str
                                })
# Общая информация
hdf.info()
hdf.describe()

In [ ]:
hdf[['had_surgery', 'age', 'outcome', 'cp_data']].value_counts()
# Смотрим на статичные показатели - видим что большинство лошадей взрослые, без патологии, выжили.

In [ ]:
hdf[['had_surgery', 'surgical_lesion']].value_counts()
# Наличие операции и поражения имеет высокую корреляцию

In [ ]:
hdf[['age', 'outcome']].value_counts()
# Смертность юных выше, чем взрослых особей

#### 2 Выбросы

In [ ]:
# Смотрим на статистику дискретных показателей
hdf[['rectal_temp', 'pulse', 'respiratory_rate', 'nasogastric_reflux_PH', 'packed_cell_volume', 'total_protein', 'abdomcentesis_total_protein']].describe()

In [ ]:
# Для каждого считаем выбросы
def outliers(col_name):
    q1 = hdf[col_name].quantile(0.25)
    q3 = hdf[col_name].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)
    return pd.concat([hdf, hdf[(~hdf[col_name].isna())&(hdf[col_name].between(lower_bound, upper_bound, inclusive=True))].sort_values(col_name)]).drop_duplicates(keep=False)[~hdf[col_name].isna()][col_name]


for x in ['rectal_temp', 'pulse', 'respiratory_rate', 'nasogastric_reflux_PH', 'packed_cell_volume', 'total_protein', 'abdomcentesis_total_protein']:
    print(f'\n{x}\n')
    print(hdf[x].quantile([0.25, 0.5, 0.75]))
    print(outliers(x).value_counts())

In [ ]:
# Отсюда видим: total_protein, nasogastric_reflux_PH - без выбросов.
# packed_cell_volume незначительно количество выбросов (1%) - можно удалить/пренебречь, в pulse аналогично явно зашкаливающие значения, хоть их и немного больше
hdf.loc[hdf.packed_cell_volume >= 74.0, 'packed_cell_volume'] = None
hdf.loc[hdf.pulse > 150, 'pulse'] = None
# rectal_temp на вид не являются аномалиями - нормальные данные
# respiratory_rate достаточно много выбросов, вероятно возможные значения, не вижу закономерности/явной ошибки.
# Аналогично с abdomcentesis_total_protein - допустимо, что у некоторых показатель завышен

#### 3 Пропуски

In [ ]:
# nasogastric_reflux_PH отсутствует у большинства респондентов по причине отсутствия показателя nasogastric_reflux (null/1).
hdf[hdf.nasogastric_reflux_PH.isna()]['nasogastric_reflux'].value_counts()
# Где null - видимо не исследовалось, заполнять нечем. Для "greater amount of reflux" - т.е. nasogastric_reflux = 2 и 3 можно заполнить пропуски медианным значением 
hdf['nasogastric_reflux_PH'].fillna(hdf.groupby('nasogastric_reflux')['nasogastric_reflux_PH'].transform('median'), inplace=True)


In [ ]:
# Для отсутствующих abdomen можно поставить значение 2 - other
hdf['abdomen'].fillna(2, inplace=True)
# Для всех c abdomen проставим медианное значение abdominocentesis_appearance, т.к. значения коррелируют
hdf['abdominocentesis_appearance'].fillna(hdf['abdominocentesis_appearance'].groupby(hdf.abdomen).transform('median'), inplace=True)

In [ ]:
# rectal_temp и pulse, respiratory_rate можно заполнить медианными в рамках групп. Здесь смотрим от каких категорий зависят дискретные величины
hdf[['surgical_lesion', 'age', 'had_surgery', 'abdomen', 'rectal_temp', 'pulse', 'respiratory_rate']].groupby(['abdomen', 'had_surgery', 'surgical_lesion', 'age']).median()
# Видим, что rectal_temp небольшой разброс - можно проставить общую медианную
hdf['rectal_temp'].fillna(hdf['rectal_temp'].median(), inplace=True)
# respiratory_rate и pulse кажется зависят от прочих, есть смысл группировать по всем.
hdf['respiratory_rate'].fillna(hdf.groupby(['had_surgery', 'surgical_lesion', 'age', 'abdomen'])['respiratory_rate'].transform('median'), inplace=True)
hdf['pulse'].fillna(hdf.groupby(['had_surgery', 'surgical_lesion', 'age', 'abdomen'])['pulse'].transform('median'), inplace=True)
# rectal_examination отсутствует, т.к. вероятно исследования не проводились. Аналогично abdomcentesis_total_protein


In [ ]:
hdf.info()
hdf.describe()

## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.